In [10]:
!nvidia-smi

Wed Oct 16 20:51:50 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 537.32                 Driver Version: 537.32       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 4060 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   44C    P8               3W / 125W |    562MiB /  8188MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

## 5.6.1计算设备

In [2]:
import torch
from torch import nn

torch.device('cpu'),torch.device('cuda'),torch.device('cuda:1')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [3]:
torch.cuda.device_count()

1

In [7]:
def try_gpu(i=0):   #@save
    """如果存在，则返回gpu(i),否则返回cpu()"""
    if torch.cuda.device_count() >= i+1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():      #@save
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices=[torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]

try_gpu(),try_gpu(10),try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## 5.6.2张量与GPU

In [8]:
x=torch.tensor([1,2,3])
x.device

device(type='cpu')

### 5.6.2.1存储在GPU上

In [9]:
X=torch.ones(2,3,device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [15]:
Y=torch.randn(2,3,device=try_gpu())
Y

tensor([[ 0.0376, -1.0382, -0.4481],
        [ 0.9914, -1.3761, -1.1559]], device='cuda:0')

### 5.6.2.2复制

In [13]:
Z=X.cuda(0)
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [16]:
Y+Z

tensor([[ 1.0376, -0.0382,  0.5519],
        [ 1.9914, -0.3761, -0.1559]], device='cuda:0')

In [17]:
Z.cuda(0) is Z

True

## 5.6.3神经网路与GPU

In [18]:
net=nn.Sequential(nn.Linear(3,1))
net=net.to(device=try_gpu())

In [20]:
net(X)

tensor([[0.0233],
        [0.0233]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [21]:
net[0].weight.data.device

device(type='cuda', index=0)